In [2]:
#Import libraries we will use in the analysis
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import inspect
from tqdm import tqdm

In [64]:
#Set starting and ending date for dowloading data from Yahoo Finance
start = dt.datetime(2000, 1, 1).strftime('%Y-%m-%d')
end = dt.datetime.now().strftime('%Y-%m-%d')
print(start, end)

2000-01-01 2024-03-14


In [4]:
#Get the list of tickers from the S&P 500
from fetch_sp500 import tickers_sp500
tickers = tickers_sp500()
tickers = tickers[:500]

In [6]:
#Fetch and save the prices for the tickers
from fetch_sp500 import fetch_and_save_prices
prices = fetch_and_save_prices(tickers)
prices.head()

[*********************100%%**********************]  500 of 500 completed


Price       Adj Close                                                  \
Ticker              A AAL      AAPL ABBV ABNB       ABT      ACGL ACN   
Date                                                                    
2000-01-03  43.683868 NaN  0.846127  NaN  NaN  9.037448  1.277778 NaN   
2000-01-04  40.346893 NaN  0.774790  NaN  NaN  8.779237  1.270833 NaN   
2000-01-05  37.844181 NaN  0.786128  NaN  NaN  8.763097  1.388889 NaN   
2000-01-06  36.403214 NaN  0.718097  NaN  NaN  9.069725  1.375000 NaN   
2000-01-07  39.436821 NaN  0.752113  NaN  NaN  9.166553  1.451389 NaN   

Price                             ...   Volume                          \
Ticker           ADBE        ADI  ...       WY WYNN      XEL       XOM   
Date                              ...                                    
2000-01-03  16.274673  28.438272  ...   973700  NaN  2738600  13458200   
2000-01-04  14.909399  26.999634  ...  1201700  NaN   425200  14510800   
2000-01-05  15.204171  27.393772  ...  1184600  NaN   500200  17485000   
2000-01-06  15.328289  26.644892  ...  1307700  NaN   344100  19461600   
2000-01-07  16.072983  27.393772  ...  1728000  NaN   469500  16603800   

Price                                                   
Ticker         XRAY XYL      YUM ZBH     ZBRA     ZION  
Date                                                    
2000-01-03   582300 NaN  3033493 NaN  1055700  1199600  
2000-01-04   317700 NaN  3315031 NaN   522450   816100  
2000-01-05  1188000 NaN  4642602 NaN   612225  1124700  
2000-01-06   534300 NaN  3947658 NaN   263925  1112100  
2000-01-07  1401000 NaN  6063647 NaN   333900   782000  

[5 rows x 3000 columns]

In [5]:
#load the data from the saved csv file
prices = pd.read_csv('sp500_prices_from_2000-01-01_to_2024-03-14.csv', header=[0, 1], index_col=0)
prices.head()

Price       Adj Close                                                  \
Ticker              A AAL      AAPL ABBV ABNB       ABT      ACGL ACN   
Date                                                                    
2000-01-03  43.683868 NaN  0.846127  NaN  NaN  9.037448  1.277778 NaN   
2000-01-04  40.346893 NaN  0.774790  NaN  NaN  8.779237  1.270833 NaN   
2000-01-05  37.844181 NaN  0.786128  NaN  NaN  8.763097  1.388889 NaN   
2000-01-06  36.403214 NaN  0.718097  NaN  NaN  9.069725  1.375000 NaN   
2000-01-07  39.436821 NaN  0.752113  NaN  NaN  9.166553  1.451389 NaN   

Price                             ...   Volume                          \
Ticker           ADBE        ADI  ...       WY WYNN      XEL       XOM   
Date                              ...                                    
2000-01-03  16.274673  28.438272  ...   973700  NaN  2738600  13458200   
2000-01-04  14.909399  26.999634  ...  1201700  NaN   425200  14510800   
2000-01-05  15.204171  27.393772  ...  1184600  NaN   500200  17485000   
2000-01-06  15.328289  26.644892  ...  1307700  NaN   344100  19461600   
2000-01-07  16.072983  27.393772  ...  1728000  NaN   469500  16603800   

Price                                                   
Ticker         XRAY XYL      YUM ZBH     ZBRA     ZION  
Date                                                    
2000-01-03   582300 NaN  3033493 NaN  1055700  1199600  
2000-01-04   317700 NaN  3315031 NaN   522450   816100  
2000-01-05  1188000 NaN  4642602 NaN   612225  1124700  
2000-01-06   534300 NaN  3947658 NaN   263925  1112100  
2000-01-07  1401000 NaN  6063647 NaN   333900   782000  

[5 rows x 3000 columns]

In [9]:
## get AAPL data
ticker = 'AAPL'
comp_price = prices.xs(ticker, axis=1, level=1, drop_level=True)
comp_price.head()

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2000-01-03,0.846127,0.999442,1.004464,0.907924,0.936384,535796800
2000-01-04,0.774790,0.915179,0.987723,0.903460,0.966518,512377600
2000-01-05,0.786128,0.928571,0.987165,0.919643,0.926339,778321600
2000-01-06,0.718097,0.848214,0.955357,0.848214,0.947545,767972800
2000-01-07,0.752113,0.888393,0.901786,0.852679,0.861607,460734400


In [98]:
## get the data of specific period
period_price = comp_price.loc['2020-01-01':'2021-01-01']
period_price.head()

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-02,73.059425,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03,72.349136,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06,72.925613,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07,72.582642,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08,73.750244,75.797501,76.110001,74.290001,74.290001,132079200


In [99]:
period_price_oc = period_price[['Open', 'Close']]
period_price_oc.head()

Price,Open,Close
Date,,
2020-01-02,74.059998,75.087502
2020-01-03,74.287498,74.357498
2020-01-06,73.447502,74.949997
2020-01-07,74.959999,74.597504
2020-01-08,74.290001,75.797501


In [100]:
period_price_oc.columns.name = None
period_price_oc.head()

,Open,Close
Date,,
2020-01-02,74.059998,75.087502
2020-01-03,74.287498,74.357498
2020-01-06,73.447502,74.949997
2020-01-07,74.959999,74.597504
2020-01-08,74.290001,75.797501


In [101]:
period_price_oc.index.name = None
period_price_oc.head()

,Open,Close
2020-01-02,74.059998,75.087502
2020-01-03,74.287498,74.357498
2020-01-06,73.447502,74.949997
2020-01-07,74.959999,74.597504
2020-01-08,74.290001,75.797501


In [102]:
period_price_oc.index.name = 'Date'
period_price_oc.head()

,Open,Close
Date,,
2020-01-02,74.059998,75.087502
2020-01-03,74.287498,74.357498
2020-01-06,73.447502,74.949997
2020-01-07,74.959999,74.597504
2020-01-08,74.290001,75.797501


In [103]:
period_price_oc.reset_index(inplace=True)
period_price_oc.head()

,Date,Open,Close
0,2020-01-02,74.059998,75.087502
1,2020-01-03,74.287498,74.357498
2,2020-01-06,73.447502,74.949997
3,2020-01-07,74.959999,74.597504
4,2020-01-08,74.290001,75.797501


In [104]:
period_price_oc.set_index('Date', inplace=True)
period_price_oc.head()

,Open,Close
Date,,
2020-01-02,74.059998,75.087502
2020-01-03,74.287498,74.357498
2020-01-06,73.447502,74.949997
2020-01-07,74.959999,74.597504
2020-01-08,74.290001,75.797501


In [105]:
period_price_oc.reset_index(inplace=True)
period_price_oc.head()

,Date,Open,Close
0,2020-01-02,74.059998,75.087502
1,2020-01-03,74.287498,74.357498
2,2020-01-06,73.447502,74.949997
3,2020-01-07,74.959999,74.597504
4,2020-01-08,74.290001,75.797501


In [106]:
s = period_price_oc.to_string(header=True, index=False)
print(s)

      Date       Open      Close
2020-01-02  74.059998  75.087502
2020-01-03  74.287498  74.357498
2020-01-06  73.447502  74.949997
2020-01-07  74.959999  74.597504
2020-01-08  74.290001  75.797501
2020-01-09  76.809998  77.407501
2020-01-10  77.650002  77.582497
2020-01-13  77.910004  79.239998
2020-01-14  79.175003  78.169998
2020-01-15  77.962502  77.834999
2020-01-16  78.397499  78.809998
2020-01-17  79.067497  79.682503
2020-01-21  79.297501  79.142502
2020-01-22  79.644997  79.425003
2020-01-23  79.480003  79.807503
2020-01-24  80.062500  79.577499
2020-01-27  77.514999  77.237503
2020-01-28  78.150002  79.422501
2020-01-29  81.112503  81.084999
2020-01-30  80.135002  80.967499
2020-01-31  80.232498  77.377502
2020-02-03  76.074997  77.165001
2020-02-04  78.827499  79.712502
2020-02-05  80.879997  80.362503
2020-02-06  80.642502  81.302498
2020-02-07  80.592499  80.007500
2020-02-10  78.544998  80.387497
2020-02-11  80.900002  79.902496
2020-02-12  80.367500  81.800003
2020-02-13

In [107]:
narrative = '\n'.join([
    f"On {row.Date}, the stock opened at {row.Open} and closed at {row.Close}."
    for row in period_price_oc.itertuples()
])
print(narrative)

On 2020-01-02, the stock opened at 74.05999755859375 and closed at 75.0875015258789.
On 2020-01-03, the stock opened at 74.2874984741211 and closed at 74.35749816894531.
On 2020-01-06, the stock opened at 73.44750213623047 and closed at 74.94999694824219.
On 2020-01-07, the stock opened at 74.95999908447266 and closed at 74.59750366210938.
On 2020-01-08, the stock opened at 74.29000091552734 and closed at 75.79750061035156.
On 2020-01-09, the stock opened at 76.80999755859375 and closed at 77.40750122070312.
On 2020-01-10, the stock opened at 77.6500015258789 and closed at 77.5824966430664.
On 2020-01-13, the stock opened at 77.91000366210938 and closed at 79.23999786376953.
On 2020-01-14, the stock opened at 79.17500305175781 and closed at 78.16999816894531.
On 2020-01-15, the stock opened at 77.9625015258789 and closed at 77.83499908447266.
On 2020-01-16, the stock opened at 78.39749908447266 and closed at 78.80999755859375.
On 2020-01-17, the stock opened at 79.06749725341797 and cl

In [108]:
print(len(s), len(narrative))

8381 21415


In [26]:
ticker = 'AAPL' 
stock = yf.Ticker(ticker)
print(len(stock.info.keys()))
for key in stock.info.keys():
    if 'earn' in key.lower() or 'price' in key.lower() or 'financ' in key.lower() or 'revenue' in key.lower() or 'profit' in key.lower():
        print(key)

129
priceHint
priceToSalesTrailing12Months
profitMargins
priceToBook
earningsQuarterlyGrowth
enterpriseToRevenue
currentPrice
targetHighPrice
targetLowPrice
targetMeanPrice
targetMedianPrice
totalRevenue
revenuePerShare
earningsGrowth
revenueGrowth
financialCurrency


In [5]:
# API_KEY = 'UJ0Z4PLU9AB6OR62'
# EPS_data = {}
# for ticker in tqdm(tickers):
#     url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={API_KEY}'
#     try:
#         r = requests.get(url)
#         if r.status_code != 200:
#             print(f'Error fetching data for {ticker}')
#             continue
#         data = r.json()
#         EPS_data[ticker] = data
#         if len(EPS_data) == 5:
#             break # testing
#     except:
#         print(f'Error fetching data for {ticker}')
#         continue
# print(len(EPS_data))

  0%|          | 0/500 [00:00<?, ?it/s]

  1%|          | 4/500 [05:07<10:34:49, 76.79s/it] 

5


KeyError: 'AAPL'

In [7]:
# print(EPS_data.keys())
# print(EPS_data['MMM'])

dict_keys(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'])
{'symbol': 'MMM', 'annualEarnings': [{'fiscalDateEnding': '2023-12-31', 'reportedEPS': '9.24'}, {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '10.1'}, {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '10.12'}, {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '8.75'}, {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '9.16'}, {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '9.98'}, {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '8.84'}, {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '8.16'}, {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '7.68'}, {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '7.49'}, {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '6.72'}, {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '6.35'}, {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '5.96'}, {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '5.75'}, {'fiscalDateEnding': '2009-12-31', 'reportedEPS': '4.68'}, {'fiscalDateEnding': '2008-12

In [66]:
import json
with open('eps_history/data-AAPL.json', 'r') as f:
    eps_dict = json.load(f)
eps_dict.keys()

dict_keys(['symbol', 'annualEarnings', 'quarterlyEarnings'])

In [67]:
eps_dict['symbol']

'AAPL'

In [69]:
len(eps_dict['quarterlyEarnings'])

112

In [70]:
len(eps_dict['annualEarnings'])

29

In [71]:
quarterly_eps = eps_dict['quarterlyEarnings']
annual_eps = eps_dict['annualEarnings']

In [72]:
quarterly_eps[0].keys()

dict_keys(['fiscalDateEnding', 'reportedDate', 'reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage'])

In [73]:
annual_eps[0].keys()

dict_keys(['fiscalDateEnding', 'reportedEPS'])

In [74]:
quarterly_eps[0]

{'fiscalDateEnding': '2023-12-31',
 'reportedDate': '2024-02-01',
 'reportedEPS': '2.18',
 'estimatedEPS': '2.1',
 'surprise': '0.08',
 'surprisePercentage': '3.8095'}

In [96]:
quarterly_eps_df = pd.DataFrame(quarterly_eps)
quarterly_eps_df.head()

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2023-12-31,2024-02-01,2.18,2.1,0.08,3.8095
1,2023-09-30,2023-11-02,1.46,1.39,0.07,5.036
2,2023-06-30,2023-08-03,1.26,1.19,0.07,5.8824
3,2023-03-31,2023-05-04,1.52,1.43,0.09,6.2937
4,2022-12-31,2023-02-02,1.88,1.94,-0.06,-3.0928


In [97]:
start = '2020-03-14'
end = '2021-5-18'
quarterly_eps_df = quarterly_eps_df[quarterly_eps_df['fiscalDateEnding'].between(start, end)]
quarterly_eps_df.head()

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
8,2021-12-31,2022-01-27,2.1,1.89,0.21,11.1111
9,2021-09-30,2021-10-28,1.24,1.24,0,0
10,2021-06-30,2021-07-27,1.3,1.01,0.29,28.7129
11,2021-03-31,2021-04-28,1.4,0.99,0.41,41.4141
12,2020-12-31,2021-01-27,1.68,1.41,0.27,19.1489


In [85]:
structured_quarterly_eps = quarterly_eps_df.to_string(header=True, index=False)
print(structured_quarterly_eps)

fiscalDateEnding reportedDate reportedEPS estimatedEPS surprise surprisePercentage
      2020-12-31   2021-01-27        1.68         1.41     0.27            19.1489
      2020-09-30   2020-10-29        0.73          0.7     0.03             4.2857
      2020-06-30   2020-07-30        0.65         0.51     0.14             27.451
      2020-03-31   2020-04-30        0.64         0.56     0.08            14.2857


In [86]:
narrative_quarterly_eps = '\n'.join([
    f"On {row.reportedDate}, EPS was reported for fiscal date ended on {row.fiscalDateEnding}, which was {row.reportedEPS} \
compared to estimated {row.estimatedEPS}. The surprise was {row.surprise} with a percentage of {row.surprisePercentage}."
    for row in quarterly_eps_df.itertuples()
])
print(narrative_quarterly_eps)

On 2021-01-27, EPS was reported for fiscal date ended on 2020-12-31, which was 1.68 compared to estimated 1.41. The surprise was 0.27 with a percentage of 19.1489.
On 2020-10-29, EPS was reported for fiscal date ended on 2020-09-30, which was 0.73 compared to estimated 0.7. The surprise was 0.03 with a percentage of 4.2857.
On 2020-07-30, EPS was reported for fiscal date ended on 2020-06-30, which was 0.65 compared to estimated 0.51. The surprise was 0.14 with a percentage of 27.451.
On 2020-04-30, EPS was reported for fiscal date ended on 2020-03-31, which was 0.64 compared to estimated 0.56. The surprise was 0.08 with a percentage of 14.2857.


In [87]:
print(len(structured_quarterly_eps), len(narrative_quarterly_eps))

414 652
